In [115]:
import numpy as np
import pandas as pd 
import plotly.graph_objects as go
import json
import os 
import sys 
from ampel.contrib.hu.util import AmpelHealpix

In [116]:
map_dir = "/mnt/c/Users/Public/Documents/Uni/master/masterarbeit/ampel/ampel-results/gw170817"
map_name  = "gw170817.fits.gz"
ah = AmpelHealpix.AmpelHealpix(map_name=map_name, save_dir=map_dir)
ah.process_map()
trigger_time = ah.trigger_time
print(trigger_time)

2457983.0285234866


In [117]:
filename = "/mnt/c/Users/Public/Documents/Uni/master/masterarbeit/ampel/ampel-results/gw170817/TransientTable/TransientTable_gw170817.csv"
#filename = "/mnt/c/Users/Public/Documents/Uni/master/masterarbeit/ampel/ampel-results/gw170817/TransientTable_gw170817.json"

dataframe = pd.read_csv(filename, sep=";")

dataframe.head()

,Unnamed: 0,ampel_z,ampel_z_precision,ampel_dist,possis_abspeak,possis_chisq,possis_ndof,chisquares,grb_spatial,kilonovaness,...,map_dist,map_dist_unc,map_name,trigger_time,map_seed,stock,ztf_id,ra,dec,channels
0,16,0.00968,0.003,9.947793,-15.133941,326.012866,15,"{'bns_maurice_1': {'chisq': 871.7288421668756,...","({'ra': 176.79, 'dec': -39.8399, 'err': 11.58,...",20,...,38.589386,6.996864,gw170817.fits.gz,2.457983e+06,gw170817,0,ZTF17aaaaaaa,197.450342,-23.381468,gw170817
1,17,0.00968,0.003,9.947793,-15.134602,333.207744,16,"{'bns_maurice_1': {'chisq': 889.2323514034093,...","({'ra': 176.79, 'dec': -39.8399, 'err': 11.58,...",20,...,38.589386,6.996864,gw170817.fits.gz,2.457983e+06,gw170817,0,ZTF17aaaaaaa,197.450342,-23.381468,gw170817
2,8,0.00968,0.003,9.947793,-15.117357,231.672415,7,{'bns_maurice_1': {'chisq': 273.85788654062617...,"({'ra': 176.79, 'dec': -39.8399, 'err': 11.58,...",19,...,38.589386,6.996864,gw170817.fits.gz,2.457983e+06,gw170817,0,ZTF17aaaaaaa,197.450342,-23.381468,gw170817
3,6,0.00968,0.003,9.947793,-15.090498,199.575464,5,"{'bns_maurice_1': {'chisq': 272.9836313109036,...","({'ra': 176.79, 'dec': -39.8399, 'err': 11.58,...",19,...,38.589386,6.996864,gw170817.fits.gz,2.457983e+06,gw170817,0,ZTF17aaaaaaa,197.450342,-23.381468,gw170817
4,12,0.00968,0.003,9.947793,-15.131204,301.247359,11,"{'bns_maurice_1': {'chisq': 755.242052940225, ...","({'ra': 176.79, 'dec': -39.8399, 'err': 11.58,...",19,...,38.589386,6.996864,gw170817.fits.gz,2.457983e+06,gw170817,0,ZTF17aaaaaaa,197.450342,-23.381468,gw170817


In [118]:
def jsonify(string: str):
    tmp_str = string.replace("'", '"')
    tmp_str = tmp_str.replace("None", "null")
    tmp_str = tmp_str.replace("True", "true")
    tmp_str = tmp_str.replace("False", "false")

    return tmp_str

In [119]:
risedec_df = pd.DataFrame()

pos_rowslist_dict = {"bns_maurice_1": [], "bns_maurice_2": [], "bns_maurice_3": [], "bns_maurice_4": []}

rowslist = []
for k, row in dataframe.iterrows():
    dec_str = jsonify(row["rise_decline"])
    #print(tmp_str)
    dec_dict = json.loads(dec_str)
    #print(dec_dict)

    rowslist.append(dec_dict)

    pos_str = jsonify(row["chisquares"])
    pos_dict = json.loads(pos_str)
    #print(pos_dict)
    for key, value in pos_dict.items():
        value["ndet"] = dec_dict["ndet"]
        value["latest_det"] = dec_dict["jd_last"]
        value["first_det"] = dec_dict["jd_det"]
        value["kilonovaness"] = row["kilonovaness"]
        pos_rowslist_dict[key].append(value)
    #print(row["rise_decline"]["ndet"])

risedec_df = pd.DataFrame(rowslist)

pos_df_dict = {}
for key in pos_rowslist_dict:
    pos_df_dict[key] = pd.DataFrame(pos_rowslist_dict[key]).sort_values(by="latest_det")


In [120]:
fig = go.Figure()

for key in pos_df_dict.keys():
    fig.add_trace(go.Scatter(x=pos_df_dict[key]["latest_det"] - trigger_time, y = pos_df_dict[key]["kilonovaness"], name=key, mode="markers"))

fig.update_layout({"xaxis_title": "days from first detection",
                   "yaxis_title": "kilonovaness"})
fig.show()

In [121]:
fig = go.Figure()

for key in pos_df_dict.keys():
    fig.add_trace(go.Scatter(x=pos_df_dict[key]["ndet"], y = pos_df_dict[key]["kilonovaness"], name=key, mode="markers"))

fig.update_layout({"xaxis_title": "days from first detection",
                   "yaxis_title": "kilonovaness"})
fig.show()

In [122]:
fig = go.Figure()

for key in pos_df_dict.keys():
    fig.add_trace(go.Scatter(x=pos_df_dict[key]["latest_det"] - trigger_time, y = pos_df_dict[key]["chisq"]/pos_df_dict[key]["ndof"], name=key, mode="markers"))

fig.update_layout({"xaxis_title": "days from first detection",
                   "yaxis_title": "Reduced chisqu"})
fig.show()

In [123]:
fig = go.Figure()

for key in pos_df_dict.keys():
    fig.add_trace(go.Scatter(x=pos_df_dict[key]["latest_det"] - trigger_time, y = pos_df_dict[key]["obspeak_model_B"], name=key))

fig.update_layout({"xaxis_title": "days from first detection",
                   "yaxis_title": "Obspeak"})
fig.show()

In [124]:
lightcurve_file = "/mnt/c/Users/Public/Documents/Uni/master/masterarbeit/ampel/ampel-results/gw170817/lightcurve.csv"
lightcurve_df = pd.read_csv(lightcurve_file, sep=";", comment="#")
lightcurve_df.dropna(how="all", inplace=True)
lightcurve_df.tail()

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,JD,Phase,Filter,mobs,mobs_sigma,Unnamed: 5,Instrument,upper_limit,UT_time
149,149,149,119,2.457997e+06,13.50,J,NaN,NaN,21.16,NaN,True,NaN
150,150,150,130,2.457998e+06,15.47,H,NaN,NaN,20.50,NaN,True,NaN
151,151,151,138,2.457998e+06,15.47,Ks,20.23,-0.10,NaN,NaN,NaN,NaN
152,152,152,139,2.458001e+06,18.46,Ks,20.81,-0.13,NaN,NaN,NaN,NaN
153,153,153,140,2.458001e+06,18.49,Ks,20.93,-0.17,NaN,NaN,NaN,NaN


In [125]:
lightcurve_df["UT_time"] = pd.to_datetime(lightcurve_df["UT_time"])

In [126]:
tmp_lc_df = lightcurve_df[lightcurve_df["JD"].isna()].copy()

tmp_lc_df["JD"] = pd.DatetimeIndex(tmp_lc_df['UT_time']).to_julian_date()

lightcurve_df.update(tmp_lc_df)

lightcurve_df["JD"] = lightcurve_df["JD"].astype(float)
lightcurve_df = lightcurve_df.sort_values(by="JD", ascending=True)

lightcurve_df.to_csv(lightcurve_file, sep=";")

In [127]:
lightcurve_df["JD"][134:147]

134    2.457991e+06
135    2.457992e+06
136    2.457992e+06
137    2.457992e+06
138    2.457993e+06
139    2.457994e+06
142    2.457995e+06
140    2.457995e+06
141    2.457995e+06
143    2.457995e+06
144    2.457995e+06
145    2.457996e+06
146    2.457996e+06
Name: JD, dtype: float64

In [128]:
filter_bands = lightcurve_df["Filter"].str.lower().unique()

fig = go.Figure()

for filter in filter_bands:
    filter_df = lightcurve_df[lightcurve_df["Filter"] == filter].copy()
    fig.add_trace(go.Scatter(x=filter_df["JD"] - trigger_time, y=filter_df["mobs"], name=filter))

fig.update_layout({"yaxis_autorange": "reversed"})
fig.show()